## 라이브러리 불러오기

In [ ]:
import numpy as np
import sys
import gym

if "../" not in sys.path:
  sys.path.append("../") 

from time import sleep
from IPython.display import clear_output

from lib.envs.gridworld import GridworldEnv

In [ ]:
env = GridworldEnv()
env.reset()

In [ ]:
env.render()

In [ ]:
env.P[3][2]

In [ ]:
#Copy the code from the last Practice code

env.reset()

obs_log = []
cumulative_reward = []
do = []

for t in range(200):
    env.render()

    # your agent here
    action = env.action_space.sample()

    ## output : (next_state, reward, is_terminal, debug_info)
    observation, reward, done, info = env.step(action)
    
    obs_log.append(observation)
    if(t==0):
        cumulative_reward.append(reward)
    else:
        cumulative_reward.append(reward + cumulative_reward[-1])

    ## if is_terminal == True, then break for loop
    if(done):
        clear_output(wait = True)
        sleep(0.2)
        print("Episode finished after {} timesteps".format(t + 1))
        sleep(0.05)
        env.render()
        break
        
    sleep(0.2)
    clear_output(wait = True)
        
env.close()

##  앞의 step1의 policy evaluation function 그대로 가져오기

In [ ]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    iter=0
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        iter+=1
        delta = 0
        print(iter," iteration!")
        #각 state마다 기존 값 V와 새로 계산되는 v값을 비교한다(수렴할때까지)
        for s in range(env.nS):
            v = 0
            #해당 state에서 취할 수 있는 각 액션마다(policy 기반)
            for a, action_prob in enumerate(policy[s]):
                #작성_해당 state의 action마다 P[s][a]를 가져와서
                for  prob, next_state, reward, done in env.P[s][a]:
                    #v에 더해주기
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

## [To do] Make a Policy Improvement function

In [ ]:
def policy_improvement(env, policy_eval=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """

    
    def one_step_lookahead(state, V):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        #빈칸1_실습pdf의 첫번째 빨간 빈칸식
        #현재 state와 value function V가 주어졌을때,  각 액션의 기대값 구하기
        A = np.zeros(env.nA)
        
        
        
        return A

    
    #policy 초기값을 설정
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:#policy_stable이 True가 될 때까지 계속 반복
        # 위에서 만든  policy_eval함수로 V 계산하기
        V = policy_eval(policy, env, discount_factor)
        
        # Policy에 변화가 생기면 False로 바꾸어 while문 다시 반복 시행할 예정
        policy_stable = True
        
        # 각 state마다
        for s in range(env.nS):
            #빈칸2_해당 s state에서 policy상 가장 best action을 chosen_a에 저장
            
            
            #빈칸3_위의 one_step_lookahead함수를 이용해서 가장 best action을 best_a에 저장
            
            
            #chosen_a가 best_a가 아니면...
            if chosen_a != best_a:
                policy_stable = False
            #빈칸4_policy를 새로운 best_a에 맞게 update하기
            #(Hint : 우리가 원하는  best action이 3번째면, policy[a]에 [0,0,1,0]) 대입하기
            
        
        # If the policy is stable we've found an optimal policy. Return it
        if policy_stable:
            return policy, V

## Running a function and show the results

In [ ]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

Grid_policy=np.reshape(np.argmax(policy, axis=1), env.shape)
print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(Grid_policy)
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

## Check if the policy_eval function makes correct answer!

In [ ]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

# 우리가 얻어낸 최적의 Policy로 실제 Gridworld episode를 관찰해보기

In [ ]:
env.reset()

obs_log = []
cumulative_reward = []
do = []

for t in range(200):
    env.render()

    # your agent here
    action = Grid_policy[int(env.s/4)][env.s%4]

    ## output : (next_state, reward, is_terminal, debug_info)
    observation, reward, done, info = env.step(action)
    
    obs_log.append(observation)
    if(t==0):
        cumulative_reward.append(reward)
    else:
        cumulative_reward.append(reward + cumulative_reward[-1])

    ## if is_terminal == True, then break for loop
    if(done):
        clear_output(wait = True)
        sleep(0.2)
        print("Episode finished after {} timesteps".format(t + 1))
        sleep(0.05)
        env.render()
        break
        
    sleep(0.5)
    clear_output(wait = True)
        
env.close()